In [1]:
import sys
sys.path.append('../')
import torch
from torch import nn
from dataset import AEDataset
from torch.utils.data import DataLoader
from lightning_model.autoencoder import LitAE 
from lightning_model.imle import LitIMLEGenerator
from lightning_model.clip import LitTextPointCloudCLIP

In [2]:
from types import SimpleNamespace
config = {
    'enc_filters': (64, 128, 128, 256),
    'latent_dim': 128,
    'enc_bn': True,
    'dec_features': (256, 256),
    'n_pts': 256,
    'dec_bn': False,
    'noise_dim': 32,
    'num_latent': 80,
    'imle_features': (256, 512),
    'latent_loss_weight': 5,
    'shape_loss_weight': 6
}
config = SimpleNamespace(**config)

In [3]:
checkpoint = '../lightning_logs/autoencoder/checkpoints/epoch=999-step=1116000.ckpt'
autoencoder = LitAE.load_from_checkpoint(checkpoint, config=config)

In [4]:
checkpoint = '../lightning_logs/clip_model_20240405-194534/version_0/checkpoints/epoch=157-step=176328.ckpt'
clip_model = LitTextPointCloudCLIP.load_from_checkpoint(checkpoint, config=config, point_cloud_encoder=autoencoder.autoencoder.encoder)

In [5]:
autoencoder.eval().cuda()
clip_model.eval().cuda()
clip_model.device

device(type='cuda', index=0)

In [6]:
root = '..'
dataset_name = 'shapenetcorev2'
# choose split type from 'train', 'test', 'all', 'trainval' and 'val'
# only shapenetcorev2 and shapenetpart dataset support 'trainval' and 'val'

train_dataset = AEDataset(root=root, dataset_name=dataset_name, num_points=config.n_pts, split='train')
val_dataset = AEDataset(root=root, dataset_name=dataset_name, num_points=config.n_pts, split='val')
test_dataset = AEDataset(root=root, dataset_name=dataset_name, num_points=config.n_pts, split='test')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)
print("datasize:", train_dataset.__len__())
print("datasize:", val_dataset.__len__())
print("datasize:", test_dataset.__len__())

datasize: 35708
datasize: 5158
datasize: 10261


In [7]:
from dataset import shapenetcore_cat2id

label_list = [i for i in range(len(shapenetcore_cat2id))]
with torch.no_grad():
    label_latents = clip_model.compute_text_features(label_list).cuda()

In [10]:
lit_lmle_model = LitIMLEGenerator(
    config, label_latents, autoencoder
)

In [11]:
for batch in train_loader:
    with torch.no_grad():
        # label_latent = clip_model.compute_text_features(batch['label'].squeeze().tolist()).cuda()
        # real_pc_latent = autoencoder.autoencoder.encoder(batch['points_encoded'].cuda())
        # out = imle_model(label_latent, real_pc_latent)
        # print(out.shape)
        
        lit_lmle_model.training_step(batch, 0)
    break   

Time taken for forward pass:  0.14560580253601074
Time taken for imle_gen:  0.036913156509399414
Time taken for find_closest_latent:  0.24357819557189941
Time taken for decoder:  0.0007839202880859375
Time taken for loss calculation:  5.472018003463745


/home/tsa87/anaconda3/envs/imle/lib/python3.7/site-packages/lightning/pytorch/core/module.py:429: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  "You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet."


In [ ]:
import datetime
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger

date_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor="val_loss")
logger = TensorBoardLogger(save_dir=f'lightning_logs', name=f"clip_model_{date_time}")
trainer = L.Trainer(max_epochs=5000, gpus=1, logger=logger, callbacks=[checkpoint_callback])
trainer.fit(model=lit_lmle_model, train_dataloaders=train_loader, val_dataloaders=val_loader)

/home/tsa87/anaconda3/envs/imle/lib/python3.7/site-packages/lightning/pytorch/trainer/connectors/accelerator_connector.py:479: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: lightning_logs/clip_model_20240406-121226
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  |

Sanity Checking: 0it [00:00, ?it/s]

torch.Size([32, 80, 128])
torch.Size([32, 128])
torch.Size([32, 80, 128])
torch.Size([32, 128])


Training: 0it [00:00, ?it/s]

torch.Size([32, 80, 128])
torch.Size([32, 128])
torch.Size([32, 80, 128])
torch.Size([32, 128])
torch.Size([32, 80, 128])
torch.Size([32, 128])
torch.Size([32, 80, 128])
torch.Size([32, 128])
torch.Size([32, 80, 128])
torch.Size([32, 128])
torch.Size([32, 80, 128])
torch.Size([32, 128])


/home/tsa87/anaconda3/envs/imle/lib/python3.7/site-packages/lightning/pytorch/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
